In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math

n_teachers = 3
n_instances = 5000

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()


x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

y_train = keras.utils.to_categorical( y_train )
y_test = keras.utils.to_categorical( y_test )

# shuffle data
idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = [ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ]
teacher_data_y = [ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ]

# gather the student data
student_data_x = x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  
student_data_y = y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  

169017344/169001437 [==============================] - 6s 0us/step


In [2]:
# train the teacher models
def get_model():
  model = keras.models.Sequential()
  model.add( keras.layers.Conv2D( 32, 3, 2, activation='relu', input_shape=x_train.shape[ 1: ] ) )
  model.add( keras.layers.MaxPooling2D( ) )
  model.add( keras.layers.Conv2D( 16, 3, 2, activation='relu' ) )
  model.add( keras.layers.Flatten() )
  model.add( keras.layers.Dense(32, activation='relu') )
  model.add( keras.layers.Dense(100, activation='softmax') )

  model.compile( optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'] )

  return model

# list of teacher models
teacher_models = [ get_model() for _ in range( n_teachers ) ]

# train teacher models
for i, (model, x, y) in enumerate( zip( teacher_models, teacher_data_x, teacher_data_y ) ):
  print( 'teacher', i )
  model.fit( x, y, epochs=25, verbose=0 )
  print( 'test accuracy:', model.evaluate( x_test, y_test, verbose=0 )[ 1 ] )

teacher 0
test accuracy: 0.15080000460147858
teacher 1
test accuracy: 0.1526000052690506
teacher 2
test accuracy: 0.15719999372959137


In [3]:
# label the data
labels = [ teacher.predict( student_data_x ) for teacher in teacher_models ]

# preform the voting
votes = np.zeros( ( student_data_x.shape[ 0 ], 100 ), dtype=np.float )
for i in range( len( student_data_x ) ):
  for j in range( n_teachers ):
    label = np.argmax( labels[ j ][ i ] )
    votes[ i, label ] += 1
  # add the noise per class
  for j in range( 100 ):
    votes[ i, j ] += np.random.laplace(loc=0.0, scale=5 )

student_data_y = keras.utils.to_categorical( np.argmax( votes, axis=1 ) )

In [4]:
# train model
student_model = get_model()
print( 'training student model' )
student_model.fit( student_data_x, student_data_y, epochs=25, verbose=0 )
print( 'test accuracy:', student_model.evaluate( student_data_x, student_data_y, verbose=0 )[ 1 ] )

training student model
test accuracy: 0.09179999679327011


In [5]:
!pip install syft==0.2.9

     |████████████████████████████████| 433 kB 12.5 MB/s 
     |████████████████████████████████| 57 kB 5.7 MB/s 
     |████████████████████████████████| 200 kB 48.0 MB/s 
     |████████████████████████████████| 20.1 MB 1.4 MB/s 
     |████████████████████████████████| 9.0 MB 26.0 MB/s 
     |████████████████████████████████| 79 kB 7.1 MB/s 
     |████████████████████████████████| 2.0 MB 42.8 MB/s 
     |████████████████████████████████| 753.4 MB 7.2 kB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 4.0 MB 43.5 MB/s 
     |████████████████████████████████| 484 kB 46.6 MB/s 
     |████████████████████████████████| 126 kB 53.4 MB/s 
     |████████████████████████████████| 66 kB 4.4 MB/s 
     |████████████████████████████████| 1.8 MB 46.4 MB/s 
     |████████████████████████████████| 449 kB 39.2 MB/s 
     |████████████████████████████████| 73 kB 2.0 MB/s 
     |████████████████████████████████| 3.6 MB 43.8 MB/s 
     |█████████████████

In [6]:
# privacy analysis 
from syft.frameworks.torch.dp import pate


teacher_preds = np.argmax( np.array( labels ), axis=2 )
print( teacher_preds.shape )

data_dep_eps, data_indep_eps = pate.perform_analysis( teacher_preds=teacher_preds,
                                                      indices=np.argmax( votes, axis=1 ),
                                                      noise_eps=0.01,
                                                      delta=10/1500
                                                     )

print(data_dep_eps, data_indep_eps)

(3, 5000)
5.505317647047939 5.505317647048129
